<a href="https://colab.research.google.com/github/droyktton/clases_ME_IB/blob/main/ME_2025_Potts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de Potts de tres estados

En la aproximación de campo medio la pseudo energía libre por partícula da

$${\tilde f}(m,T)=
J z m^2/2 - k_B T \ln[2 e^{-Jz m/2 k_B T}+e^{Jz m/k_B T}]
$$

El problema depende solo del paramétro $t \equiv k_B T/J z$ (que a los fines prácticos es como tomar $z=J=k_B=1$), y si medimos la energía en unidades de $J z$ y definimos $f={\tilde f}/Jz$, tenemos

$$
f(m,t)=\frac{m^2}{2}-t \ln\left[2 e^{-\frac{m}{2 t}}+e^{\frac{m}{t}}\right]
$$


$$
\partial_m f(m,t)=
-\frac{\left(-{e^{-\frac{m}{2 t}} }+{e^{\frac{m}{t}}}\right) }{2 e^{-\frac{m}{2 t}}+e^{\frac{m}{t}}}+m$$


$$
\partial^2_m f(m,t)=
1-\frac{\left(\frac{e^{-\frac{m}{2 t}} }{2 t}+\frac{e^{\frac{m}{t}}}{t}\right) }{2 e^{-\frac{m}{2 t}}+e^{\frac{m}{t}}}+\frac{\left(-\frac{e^{-\frac{m}{2 t}} }{t}+\frac{e^{\frac{m}{t}} }{t}\right)^2 t}{\left(2 e^{-\frac{m}{2 t}}+e^{\frac{m}{t}}\right)^2}
$$


* Los puntos de equilibrio estable o metaestable son aquellos que $$\partial_m f(m,t)=0, \;\partial^2_m f(m,t)>0$$

* Si para algún rango de $t$ hay más de un mínimo, $m_1(t) \neq m_2(t)$, y existe una $t_c$ tal que  $f(m_1(t),t)=f(m_2(t),t)$ se cruzan en $t_c$, entonces $t_c$ es la temperatura de transición de primer orden.

* Los límites de estabilidad de $m_1$ y $m_2$, $t_1$ y $t_2$ determinan el ancho $|t_2-t_1|$ y los saltos de $|m_1(t_1)-m_2(t_1)|$, $|m_1(t_2)-m_2(t_2)|$ del ciclo de histéresis.

In [ ]:
#@title Definicion de f, df/dm y d2f/dm2

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from ipywidgets import interact, FloatSlider


# Define the function f
def f(x, T, J):
    return J*x**2/2 - T*np.log(2*np.exp(-J*x/(2*T))+np.exp(J*x/T))

# Define the function f'
def f1(x, T, J):
    # Calculate the numerator of the first term
    numerator = T * (J * np.exp(J * x / T) / T - J * np.exp(-J * x / (2 * T)) / T)

    # Calculate the denominator of the first term
    denominator = 2 * np.exp(-J * x / (2 * T)) + np.exp(J * x / T)

    # Compute the entire expression
    result = -numerator / denominator + J * x

    return result

# Define the function f''
def f2(x, T, J):
    # First term
    term1_numerator = T * (J**2 * np.exp(J * x / T) / T**2 + J**2 * np.exp(-J * x / (2 * T)) / (2 * T**2))
    term1_denominator = 2 * np.exp(-J * x / (2 * T)) + np.exp(J * x / T)
    term1 = - term1_numerator / term1_denominator

    # Second term
    term2_numerator = T * (J * np.exp(J * x / T) / T - J * np.exp(-J * x / (2 * T)) / T)**2
    term2_denominator = (2 * np.exp(-J * x / (2 * T)) + np.exp(J * x / T))**2
    term2 = term2_numerator / term2_denominator

    # Final expression
    result = term1 + term2 + J

    return result


In [ ]:
#@title Pseudo Energia libre interactiva
def plot_free_energy(t):
  j=1
  m = np.linspace(0.0 , 1.0, 1000)
  y = f(m,t,j)
  plt.plot(m, y, label=r"$F(m,t)$")

  # Define initial guesses (based on visualization or prior knowledge)
  initial_guesses = [0, 1]  # These are just guesses for local minima

  minima = []
  for guess in initial_guesses:
      result = minimize(lambda x: f(x, t, 1), x0=guess)
      minima.append((result.x, result.fun))

  # Print the found minima
  for i, (x_min, f_min) in enumerate(minima):
      print(f"Local minimum {i+1} at x = {x_min[0]}, f(x) = {f_min}")
      fre=f(x_min[0],t,1)
      plt.scatter(x_min[0], fre, color='yellow', s=50, label=f'Point (M={x_min[0]}, f={fre})')

  plt.show()


#T critica analitica
Tc=(3./8.)/np.log(2.)

interact(plot_free_energy,
         t=FloatSlider(value=Tc, min=0.0, max=0.75, step=0.0001, description='Temperature', readout_format='.5f')
         )




## Verificación del resultado exacto para $T_c$ de primer orden y salto de magnetización

Se cononce la solución exacta para la transición de primer order. Notar que $m=0$ es siempre un extremo para cualquier $T$. El otro extremo depende de $T$, y hay una $T_c$ donde la pseudo energía libre es igual para ambos mínimos, y ambos son estables.

La solución conocida es $T_c=(3/8)/\ln(2)$, con $m=0$ y $m=1/2$. No vamos a derivarlo pero vamos a chequear que funciona.


In [ ]:
#@title energía libre y derivadas de la solución $m=0$ a $T=(3/8)/\ln(2)$
print("f(m=0,Tc,1)=",f(0,(3/8)/np.log(2),1))
print("df(m=0,Tc,1)/dm=",f1(0,(3/8)/np.log(2),1))
print("d2f(m=0,Tc,1)/dm2=",f2(0,(3/8)/np.log(2),1))

In [ ]:
#@title energía libre y derivadas de la solución $m=1/2$ a $T=(3/8)/\ln(2)$
print("f(m=1/2,Tc,1)=",f(0,(3/8)/np.log(2),1))
print("df(m=1/2,Tc,1)/dm=",f1(0.5,(3/8)/np.log(2),1))
print("d2f(m=1/2,Tc,1)/dm2=",f2(0.5,(3/8)/np.log(2),1))

## Pseudo energía libre de Landau

Desarrollo en serie de la pseudo energía de campo medio.

In [ ]:
import sympy as sp
from sympy import init_printing

# Initialize pretty printing in LaTeX
init_printing(use_latex='mathjax')

# Define the symbols
J, z, m, T, k_B = sp.symbols('J z m T k_B')

# Define the expression
expr = J * z * m**2 / 2 - k_B * T * sp.ln(2 * sp.exp(-J * z * m / (2 * k_B * T)) + sp.exp(J * z * m / (k_B * T)))

# Expand the expression in a series as a function of m, up to the fourth order
series_expr = expr.series(m, 0, 5)  # Expanding up to the fourth order (m^4)

# Display the LaTeX formatted output directly in Jupyter
print("Pseudo Energía Libre de campo medio:")
display(expr)

print("\n\nPseudo Energía Libre de Landau:")
display(series_expr)

Vemos entonces que el coeficiente cuadrático cambia de signo cuando

$$\frac{Jz}{2}=\frac{J^2 z^2}{4 T k_B}$$

es decir cuando

$$k_B T=\frac{J z}{2}$$

mientras que el coeficiente del orden cúbico es negativo, y el del cuártico positivo.

Por simplicidad y sin pérdida de generalidad, para visualizar podemos escribir

$$\tilde{f}=m^2 (T-1)/2 - y m^3/3 + m^4/4$$

con $y>1$. Ya vimos que esta energía libre tiene una transición de primer orden.

## Límites de metastabilidad e histéresis

Dibujando la curva de nivel df/dm=0, y marcando cual solución corresponde a un mínimo (local o global) podemos ver que hay histeresis.

Del gráfico podemos apreciar que

* Las temperaturas límites son
$$T_1 \approx 0.546$$
$$T_2=0.500$$

* Y los saltos de magnetización
$$M_1 \approx 0.374$$
$$M_2 \approx 0.76$$


In [ ]:
#@title función para dibujar los extremos (curva de nivel df/dm=0)
import numpy as np

def plot_contour(j):
  # Create a grid of (x, y) points
  t = np.linspace(0.1, 0.75, 2000)
  m = np.linspace(-0.1 , 0.999, 2000)
  M, T = np.meshgrid(m, t)

  # Evaluate the function on the grid
  Z = f1(M, T, j)

  # second derivative
  Z2 = f2(M, T,j)

  stable = Z2 > 0
  unstable = Z2 < 0

  filtered_Z = np.where(stable,Z,np.nan)
  plt.contour(M, T, filtered_Z, levels=[0], colors='blue')

  filtered_Z = np.where(unstable,Z,np.nan)
  plt.contour(M, T, filtered_Z, levels=[0], colors='red',linestyles='--')

  T1=0.546
  M1=0.374
  plt.scatter(M1, T1, color='yellow', s=50, label=f'Sobrecalentado (M1={M1}, T1={T1})')
  #plt.legend()
  plt.axhline(y=T1, color='y', linestyle='--', xmin=0, xmax=M1)
  T2=0.5
  M2=0.72
  plt.axhline(y=T2, color='y', linestyle='--', xmin=0, xmax=M2)
  plt.scatter(M2, T2, color='yellow', s=50, label=f'Sobreenfriado (M2={M2}, T2={T2})')


  #print(stable.shape)

  # Customize plot
  plt.title(r"Contorno de $df/dm =  0$")
  plt.xlabel("M")
  plt.ylabel("T")
  plt.grid(True)

  Tc=(3./8)/np.log(2.0)
  Mc=0.5
  #plt.axhline(y=Tc, color='g', linestyle='--', xmin=0, xmax=0.5)
  plt.scatter(Mc, Tc, color='r', s=50, label=f'Transición (Mc={Mc}, Tc={Tc})')
  plt.legend()

  plt.show()

plot_contour(1)


En el dibujo de arriba, las lineas azules son estables o metaestables. Las rojas a rayas son extremos inestables.

* Cuando venimos de $T$ alta, siempre venimos por $M=0$, el único mínimo.

* **Equilibrio** A $T=T_c \approx 0.541$ hay una transición termodinámica donde $M$ salta a $M=0.5$ exactamente, y leugo sigue aumentando bajando $T$. Si volvemos a subir $T$, ocurre el mismo salto en sentido inverso, y todo es reversible si hacemos todo muy lentamente. *No hay histeresis* si en cada momento estamos en el mínimo global de la energía libre de Landau.

* **Histéresis, metastabilidad, sobreenfriado, sobrecalentado** Si no hacemos todo lentamente, cuando venimos de $T$ alta, venimos por $M=0$ y el salto de $M \approx 0.72$ se produce a $T_2=0.5 < T_c$, sobreenfriando la fase desordenada. Cuando subimos $T$ de nuevo, un salto inverso de magnitud $|M|\approx 0.374$ ocurre a $T_1\approx 0.546$, sobrecalentando la fase ordenada.   


